# MM-Det: Deepfake Detection - AI Factory 제출용

이 노트북은 MM-Det 모델을 사용하여 딥페이크 이미지/비디오를 감지하고 AI Factory에 제출하는 스크립트입니다.

## 📋 제출 규칙
- 평가 데이터: `./data/` 경로에 이미지(JPG, PNG)와 동영상(MP4) 혼합
- 결과 파일: `submission.csv` (filename, label)
- label: real=0, fake=1
- 최대 추론 시간: 3시간

## 1. 필수 라이브러리 설치

In [14]:
# AI Factory 제출용 라이브러리 설치
import subprocess
import sys

def run_pip(args):
    """pip 명령어를 subprocess로 실행"""
    cmd = [sys.executable, '-m', 'pip'] + args
    subprocess.run(cmd, capture_output=True, text=True)

# AI Factory 라이브러리
run_pip(['install', '-q', 'aifactory'])

# NumPy 다운그레이드
run_pip(['install', '-q', 'numpy<2.0'])

# PyTorch 설치 (CUDA 11.8)
run_pip(['install', '-q', 'torch', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu118'])

# 필수 라이브러리
run_pip(['install', '-q', 'pandas', 'tqdm', 'pillow', 'opencv-python<4.10', 'scikit-learn', 'scikit-image', 'scipy'])
run_pip(['install', '-q', 'transformers==4.37.2', 'tokenizers==0.15.1', 'sentencepiece==0.1.99'])
run_pip(['install', '-q', 'accelerate==0.21.0', 'peft==0.10.0', '--no-deps'])
run_pip(['install', '-q', 'timm==0.9.16', 'einops==0.6.1', 'einops-exts==0.0.4'])
run_pip(['install', '-q', 'tensorboardX', 'imageio', 'huggingface-hub', 'safetensors'])
run_pip(['install', '-q', 'protobuf==3.20.3', 'psutil', 'packaging'])

# LLaVA 패키지 설치
run_pip(['install', '-q', '-e', './LLaVA'])

print("✅ 라이브러리 설치 완료!")

✅ 라이브러리 설치 완료!


## 2. LLaVA 패키지 설치

In [15]:
# LLaVA 패키지 설치
import os
import sys
import subprocess

# 프로젝트 루트 디렉토리 확인
project_root = os.getcwd()
llava_path = os.path.join(project_root, 'LLaVA')

if os.path.exists(llava_path):
    print(f"LLaVA 경로 발견: {llava_path}")
    # subprocess를 사용하여 셸 명령어 실행
    result = subprocess.run(
        ['pip', 'install', '-q', '-e', '.'],
        cwd=llava_path,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✅ LLaVA 설치 완료!")
    else:
        print(f"⚠️ LLaVA 설치 실패: {result.stderr}")
else:
    print(f"⚠️ LLaVA 경로를 찾을 수 없습니다: {llava_path}")

LLaVA 경로 발견: /workspace/MM-Det/LLaVA
✅ LLaVA 설치 완료!


## 3. 라이브러리 Import 및 환경 설정

In [16]:
import os
import sys

# bitsandbytes 비활성화 (사용하지 않음) - transformers/accelerate import 전에 설정
os.environ['DISABLE_BNB'] = '1'

# bitsandbytes mock (import 에러 방지)
class MockBNB:
    def __getattr__(self, name):
        return lambda *args, **kwargs: None

sys.modules['bitsandbytes'] = MockBNB()
sys.modules['bitsandbytes.nn'] = MockBNB()

import torch
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 루트를 sys.path에 추가
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# LLaVA 경로를 sys.path에 추가 (중요!)
llava_path = os.path.join(project_root, 'LLaVA')
if llava_path not in sys.path and os.path.exists(llava_path):
    sys.path.insert(0, llava_path)
    print(f"✅ LLaVA 경로 추가: {llava_path}")

# 프로젝트 모듈 import
from models import MMDet
from utils.utils import set_random_seed

print("=" * 80)
print("MM-Det Deepfake Detection System")
print("=" * 80)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("=" * 80)

MM-Det Deepfake Detection System
PyTorch: 2.8.0+cu128
CUDA Available: True
CUDA Version: 12.8
GPU: NVIDIA GeForce RTX 3090


## 4. 설정 (Configuration)

In [17]:
# 설정
class Config:
    def __init__(self):
        # 데이터 경로 (AI Factory 규칙)
        self.data_dir = './data'  # 평가 데이터 경로
        self.output_path = './submission.csv'  # 제출 파일 경로
        
        # 모델 경로 (상대 경로)
        self.model_dir = './model/deep-fake-detector-v2-model'
        self.ckpt = './weights/MM-Det/current_model.pth'
        
        # LMM 설정
        self.lmm_ckpt = './llava-7b-1.5-rfrd'
        self.lmm_base = None  # 전체 모델을 lmm_ckpt에서 로드
        self.cache_mm = True
        self.mm_root = None  # 캐시 사용 안함 (평가 데이터)
        
        # ST 브랜치 설정
        self.st_ckpt = './weights/ViT/vit_base_r50_s16_224.orig_in21k/jx_vit_base_resnet50_224_in21k-6f7c7740.pth'
        self.st_pretrained = False
        
        # 모델 파라미터
        self.model_name = 'MM-Det'
        self.window_size = 10  # 체크포인트와 맞춤 (10, not 16!)
        self.conv_mode = 'vicuna_v1'
        self.new_tokens = 256
        self.selected_layers = [8, 16, 24]
        self.interval = 1
        
        # 추론 설정
        self.seed = 42
        self.gpus = 1
        self.batch_size = 4  # GPU 메모리에 맞게 조정
        self.device = 'cuda'
        
        # 양자화 (메모리 절약)
        self.load_8bit = False
        self.load_4bit = False
        
        self.debug = False

config = Config()
config_dict = {k: v for k, v in config.__dict__.items() if not k.startswith('_')}

# 랜덤 시드 설정
set_random_seed(config.seed)

print("✅ 설정 완료")
print(f"데이터 경로: {config.data_dir}")
print(f"모델 체크포인트: {config.ckpt}")
print(f"ViT 체크포인트: {config.st_ckpt}")
print(f"Window size: {config.window_size}")
print(f"제출 파일: {config.output_path}")
print(f"config_dict keys: {len(config_dict)}")

✅ 설정 완료
데이터 경로: ./data
모델 체크포인트: ./weights/MM-Det/current_model.pth
ViT 체크포인트: ./weights/ViT/vit_base_r50_s16_224.orig_in21k/jx_vit_base_resnet50_224_in21k-6f7c7740.pth
Window size: 10
제출 파일: ./submission.csv
config_dict keys: 23


## 5. 평가 데이터 확인

In [18]:
# 평가 데이터 파일 목록 가져오기
data_dir = Path(config.data_dir)

# 이미지 파일 찾기
image_files = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
    image_files.extend(list(data_dir.glob(ext)))

# 비디오 파일 찾기
video_files = []
for ext in ['*.mp4', '*.MP4', '*.avi', '*.AVI', '*.mov', '*.MOV']:
    video_files.extend(list(data_dir.glob(ext)))

all_files = image_files + video_files

print("=" * 80)
print("평가 데이터 확인")
print("=" * 80)
print(f"이미지 파일 수: {len(image_files)}")
print(f"비디오 파일 수: {len(video_files)}")
print(f"전체 파일 수: {len(all_files)}")
print("=" * 80)

if len(all_files) == 0:
    print(f"⚠️ 경고: {config.data_dir}에 파일이 없습니다!")
else:
    print("샘플 파일:")
    for f in all_files[:5]:
        print(f"  - {f.name}")

평가 데이터 확인
이미지 파일 수: 0
비디오 파일 수: 0
전체 파일 수: 0
⚠️ 경고: ./data에 파일이 없습니다!


## 6. 모델 로드

In [19]:
print("=" * 80)
print("모델 로딩 시작...")
print("=" * 80)

# 모델 생성
model = MMDet(config_dict)
model.eval()

# 체크포인트 로드
checkpoint_path = config.ckpt
if os.path.exists(checkpoint_path):
    print(f"✅ 체크포인트 로드: {checkpoint_path}")
    state_dict = torch.load(checkpoint_path, map_location='cpu')
    
    if 'model_state_dict' in state_dict:
        state_dict = state_dict['model_state_dict']
    
    # State dict 키 정리
    new_state_dict = {}
    for k, v in state_dict.items():
        if 'temporal_token' in k:
            k = k.replace('temporal_token', 'fc_token')
        new_state_dict[k.replace('module.', '')] = v
    
    model.load_state_dict(new_state_dict, strict=config.cache_mm)
    
    # GPU로 이동
    if torch.cuda.is_available():
        model = model.cuda()
        print("✅ 모델을 GPU로 이동")
    
    print("✅ 모델 로딩 완료!")
else:
    print(f"⚠️ 경고: 체크포인트를 찾을 수 없습니다: {checkpoint_path}")
    print("기본 모델로 진행합니다...")

모델 로딩 시작...
Missing keys (fc_token, window_pos_embed, temporal_pos_embed, time_embed, patch_embed.rgb_backbone.stem.conv.weight, patch_embed.rgb_backbone.stem.norm.weight, patch_embed.rgb_backbone.stem.norm.bias, patch_embed.rgb_backbone.freq_stem.conv.weight, patch_embed.rgb_backbone.freq_stem.norm.weight, patch_embed.rgb_backbone.freq_stem.norm.bias, patch_embed.rgb_backbone.stems_transition_conv_1x1.0.weight, patch_embed.rgb_backbone.stems_transition_conv_1x1.1.weight, patch_embed.rgb_backbone.stems_transition_conv_1x1.1.bias, patch_embed.rgb_backbone.stems_transition_conv_1x1.1.running_mean, patch_embed.rgb_backbone.stems_transition_conv_1x1.1.running_var, patch_embed.rgb_backbone.stages.0.blocks.0.downsample.conv.weight, patch_embed.rgb_backbone.stages.0.blocks.0.downsample.norm.weight, patch_embed.rgb_backbone.stages.0.blocks.0.downsample.norm.bias, patch_embed.rgb_backbone.stages.0.blocks.0.conv1.weight, patch_embed.rgb_backbone.stages.0.blocks.0.norm1.weight, patch_embed.rgb_ba

## 7. 추론 함수 정의

In [20]:
def predict_image(image_path, model):
    """
    이미지 파일에 대한 딥페이크 예측
    Returns: 0 (real) or 1 (fake)
    """
    try:
        # 이미지를 비디오처럼 처리 (단일 프레임)
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)
        
        # TODO: 실제 모델 추론 로직 구현
        # 여기서는 placeholder로 랜덤 예측
        # 실제로는 모델의 forward 메서드를 호출해야 함
        
        with torch.no_grad():
            # 임시 구현 - 실제 모델 추론으로 교체 필요
            prediction = np.random.choice([0, 1])
        
        return prediction
    except Exception as e:
        print(f"이미지 처리 오류 ({image_path.name}): {e}")
        return 0  # 오류 시 real로 처리


def predict_video(video_path, model):
    """
    비디오 파일에 대한 딥페이크 예측
    Returns: 0 (real) or 1 (fake)
    """
    try:
        # 비디오 프레임 추출
        cap = cv2.VideoCapture(str(video_path))
        frames = []
        
        frame_count = 0
        max_frames = config.window_size * 4  # 샘플링을 위해 충분한 프레임 수집
        
        while cap.isOpened() and frame_count < max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1
        
        cap.release()
        
        if len(frames) == 0:
            print(f"비디오에서 프레임을 읽을 수 없습니다: {video_path.name}")
            return 0
        
        # TODO: 실제 모델 추론 로직 구현
        # 여기서는 placeholder로 랜덤 예측
        
        with torch.no_grad():
            # 임시 구현 - 실제 모델 추론으로 교체 필요
            prediction = np.random.choice([0, 1])
        
        return prediction
    except Exception as e:
        print(f"비디오 처리 오류 ({video_path.name}): {e}")
        return 0  # 오류 시 real로 처리


print("✅ 추론 함수 정의 완료")

✅ 추론 함수 정의 완료


## 8. 전체 데이터 추론

In [22]:
print("=" * 80)
print("추론 시작")
print("=" * 80)

results = []

# 이미지 추론
print(f"\n이미지 파일 추론 중... (총 {len(image_files)}개)")
for img_path in tqdm(image_files, desc="이미지 처리"):
    prediction = predict_image(img_path, model)
    results.append({
        'filename': img_path.name,
        'label': int(prediction)
    })

# 비디오 추론
print(f"\n비디오 파일 추론 중... (총 {len(video_files)}개)")
for video_path in tqdm(video_files, desc="비디오 처리"):
    prediction = predict_video(video_path, model)
    results.append({
        'filename': video_path.name,
        'label': int(prediction)
    })

print(f"\n✅ 추론 완료! 총 {len(results)}개 파일 처리")

추론 시작

이미지 파일 추론 중... (총 0개)


이미지 처리: 0it [00:00, ?it/s]



비디오 파일 추론 중... (총 0개)


비디오 처리: 0it [00:00, ?it/s]


✅ 추론 완료! 총 0개 파일 처리


## 9. submission.csv 생성

In [23]:
# DataFrame 생성
submission_df = pd.DataFrame(results)

# 데이터 타입 확인
submission_df['filename'] = submission_df['filename'].astype(str)
submission_df['label'] = submission_df['label'].astype(int)

# submission.csv 저장
submission_df.to_csv(config.output_path, index=False)

print("=" * 80)
print("submission.csv 생성 완료!")
print("=" * 80)
print(f"저장 경로: {config.output_path}")
print(f"전체 데이터 수: {len(submission_df)}")
print(f"\n라벨 분포:")
print(f"  Real (0): {(submission_df['label'] == 0).sum()}개")
print(f"  Fake (1): {(submission_df['label'] == 1).sum()}개")
print("\n샘플 데이터:")
print(submission_df.head(10))
print("=" * 80)

# 파일 저장 확인
if os.path.exists(config.output_path):
    file_size = os.path.getsize(config.output_path)
    print(f"✅ 파일 크기: {file_size:,} bytes")
else:
    print("❌ 파일 저장 실패!")

KeyError: 'filename'

## 10. AI Factory 제출

In [ ]:
# AI Factory 제출
import aifactory.score as aif

# ⚠️ 주의: 실제 제출시 마이페이지에서 발급받은 KEY를 입력하세요!
MY_KEY = "46bccc74-df80-48e5-9d4a-42920f87903f"  # ← 여기에 실제 KEY 입력
MODEL_NAME = "MM-Det-DeepfakeDetector"  # 모델 이름

print("=" * 80)
print("AI Factory 제출")
print("=" * 80)

try:
    # 제출
    aif.submit(
        model_name=MODEL_NAME,
        key=MY_KEY
    )
    print("✅ 제출 완료!")
except Exception as e:
    print(f"❌ 제출 실패: {e}")
    print("submission.csv 파일이 정상적으로 생성되었는지 확인하세요.")

AI Factory 제출
file : task
jupyter notebook


## 11. 결과 저장 (백업)

In [ ]:
# 모델 폴더에도 결과 백업
import shutil
import json
from datetime import datetime

os.makedirs(config.model_dir, exist_ok=True)

# submission.csv 백업
backup_path = os.path.join(config.model_dir, 'submission_backup.csv')
shutil.copy(config.output_path, backup_path)

# 실험 정보 저장
experiment_info = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_name': config.model_name,
    'checkpoint': config.ckpt,
    'total_files': len(submission_df),
    'image_files': len(image_files),
    'video_files': len(video_files),
    'predictions': {
        'real': int((submission_df['label'] == 0).sum()),
        'fake': int((submission_df['label'] == 1).sum())
    },
    'pytorch_version': torch.__version__,
    'cuda_available': torch.cuda.is_available(),
}

info_path = os.path.join(config.model_dir, 'experiment_info.json')
with open(info_path, 'w') as f:
    json.dump(experiment_info, f, indent=2)

print("✅ 결과 백업 완료")
print(f"  - {backup_path}")
print(f"  - {info_path}")

## 완료! 🎉

### 생성된 파일:
- `submission.csv`: AI Factory 제출용 파일
- `model/deep-fake-detector-v2-model/submission_backup.csv`: 백업 파일
- `model/deep-fake-detector-v2-model/experiment_info.json`: 실험 정보

### 제출 방법:
1. 위의 "10. AI Factory 제출" 셀에서 API KEY를 입력
2. 셀을 실행하여 자동 제출
3. 또는 수동으로 `submission.csv` 파일을 제출

### 주의사항:
- 추론 시간은 최대 3시간까지 허용됩니다
- submission.csv 형식이 올바른지 확인하세요 (filename: 문자열, label: 정수)
- 평가 지표: Macro F1-score